In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.5 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import LongformerModel, LongformerTokenizer
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import DistilBertModel, DistilBertTokenizerFast
import tensorflow as tf
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from transformers import EarlyStoppingCallback
from torch.optim import AdamW

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Thesis/commentbig5sample.csv", encoding='latin-1')

In [5]:
# Drop colums
df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

In [6]:
df.head()

author                                               body  \
0  MetricExpansion  Those stats come test Echoing comment I made r...   
1         cyanisis  Ok ok op Some advice just dont Set direct depo...   
2      BadgerKid96  It seem like went pretty fast I look back it A...   
3    Silverlight42                   ok whos using force powers again   
4     pseudomuffin  Will recommendworks well name brand I use time...   

   agreeableness  openness  conscientiousness  extraversion  neuroticism  \
0           0.30      0.70               0.15          0.15         0.50   
1           0.09      0.59               0.05          0.72         0.07   
2           0.77      0.73               0.73          0.01         0.98   
3           0.09      0.61               0.13          0.04         0.72   
4           0.79      0.84               0.86          0.53         0.01   

   count  word_count  
0   2768          77  
1  15648          47  
2   2319          36  
3  43035           6  
4   3337          10

SyntaxError: ignored

In [8]:

# # Initialize tokenizer

# tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

# Preprocess tweets: tokenize and pad/truncate
max_length = 64

df['body'] = df['body'].apply(lambda x: tokenizer.encode_plus(x, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt'))



# Split into train and test first
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

# Split df_train into train and validation
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)

In [9]:
# Define a PyTorch dataset
class PersonalityDataset(Dataset):
    def __init__(self, tweets, targets):
        self.tweets = tweets
        self.targets = targets

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
      input_ids = torch.tensor(self.tweets[idx]['input_ids'], dtype=torch.long).squeeze(0)
      attention_mask = torch.tensor(self.tweets[idx]['attention_mask'], dtype=torch.long).squeeze(0)
      targets = torch.tensor(self.targets[idx], dtype=torch.float)

      return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'targets': targets
      }

In [11]:
class DistilBERTForPersonalityTraits(torch.nn.Module):
    def __init__(self):
        super(DistilBERTForPersonalityTraits, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-cased')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 5)  # Update the input size to match the DistilBERT hidden size

    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token representation
        output = self.dropout(pooled_output)
        output = self.linear(output)
        return output


In [12]:
# Prepare data loaders
batch_size = 16
train_dataset = PersonalityDataset(df_train['body'].tolist(), df_train[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
val_dataset = PersonalityDataset(df_val['body'].tolist(), df_val[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
test_dataset = PersonalityDataset(df_test['body'].tolist(), df_test[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Instantiate the model and define optimizer and loss function

model = DistilBERTForPersonalityTraits()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.MSELoss()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
train_dataset.size


AttributeError: ignored

In [ ]:
# Training loop
epochs = 10

# Early stopping parameters
patience = 3
best_val_loss = float('inf')
patience_counter = 0
accumulation_steps = 16

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad() if i % accumulation_steps == 0 else None

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

# Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)

        # Backward pass
        loss = loss / accumulation_steps
        loss.backward()
        if (i+1) % accumulation_steps == 0 or i+1 == len(train_loader):  # update on last step even if accumulation steps have not been reached
            optimizer.step()
            optimizer.zero_grad()  # make sure to reset gradient after updating

        total_loss += loss.item()

        # Delete unnecessary tensors to save GPU memory
        del input_ids
        del attention_mask
        del targets
        del outputs
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Train loss at epoch {epoch + 1}: {avg_train_loss}")

    # Validation loop
    model.eval()
    val_total_loss = 0
    all_predictions = [[] for _ in range(5)]  # Separate predictions for each trait
    all_targets = [[] for _ in range(5)]  # Separate targets for each trait

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs, targets)

            val_total_loss += loss.item()

            # Split predictions and targets for each trait
            for i in range(5):
                all_predictions[i].extend(outputs[:, i].cpu().numpy())
                all_targets[i].extend(targets[:, i].cpu().numpy())

    avg_val_loss = val_total_loss / len(val_loader)
    print(f"Validation loss at epoch {epoch + 1}: {avg_val_loss}")

    # Check if early stopping conditions are met
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0  # reset counter
        # save the best model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1  # increment counter
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break  # break out of the training loop


# After the validation loop...
trait_names = ["Extroversion", "Neuroticism", "Agreeableness", "Conscientiousness", "Openness"]

for i in range(5):
    predictions = all_predictions[i]
    targets = all_targets[i]

    mse = mean_squared_error(targets, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(targets, predictions)

    print(f"For {trait_names[i]}:")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"R^2: {r2}\n")



<ipython-input-9-217b49c1381b>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(self.tweets[idx]['input_ids'], dtype=torch.long).squeeze(0)
<ipython-input-9-217b49c1381b>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(self.tweets[idx]['attention_mask'], dtype=torch.long).squeeze(0)


Train loss at epoch 1: 0.028679456053809685
Validation loss at epoch 1: 0.29812896038804737
Train loss at epoch 2: 0.01628250875933604
Validation loss at epoch 2: 0.15516419016889163
Train loss at epoch 3: 0.009738689999688755
Validation loss at epoch 3: 0.11970942573887962
Train loss at epoch 4: 0.00899244403805245
Validation loss at epoch 4: 0.1188946281160627
Train loss at epoch 5: 0.008669018559157849
Validation loss at epoch 5: 0.11196909791656903
Train loss at epoch 6: 0.008505696676332842
Validation loss at epoch 6: 0.10492613751973424
Train loss at epoch 7: 0.007734304522587494
Validation loss at epoch 7: 0.10198103051100459
Train loss at epoch 8: 0.007506690555336801
Validation loss at epoch 8: 0.10251173004508018
Train loss at epoch 9: 0.007316194110634652
Validation loss at epoch 9: 0.10342484286853246
Train loss at epoch 10: 0.007388883151791313
Validation loss at epoch 10: 0.1026695647409984
Early stopping triggered.
For Extroversion:
MSE: 0.11422082036733627
RMSE: 0.33796

IndentationError: ignored

In [ ]:
# Evaluation on test set
model.eval()
test_total_loss = 0
all_predictions = [[] for _ in range(5)]  # Separate predictions for each trait
all_targets = [[] for _ in range(5)]  # Separate targets for each trait

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)

        test_total_loss += loss.item()

        # Split predictions and targets for each trait
        for i in range(5):
            all_predictions[i].extend(outputs[:, i].cpu().numpy())
            all_targets[i].extend(targets[:, i].cpu().numpy())

trait_names = ["Ext", "Neu", "Agre", "Con", "Ope"]

for i in range(5):
    predictions = all_predictions[i]
    targets = all_targets[i]

    mse = mean_squared_error(targets, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(targets, predictions)
    r2 = r2_score(targets, predictions)

    print(f"For {trait_names[i]}:")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R^2: {r2}\n")

<ipython-input-7-217b49c1381b>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(self.tweets[idx]['input_ids'], dtype=torch.long).squeeze(0)
<ipython-input-7-217b49c1381b>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(self.tweets[idx]['attention_mask'], dtype=torch.long).squeeze(0)


For Ext:
MSE: 0.09643164277076721
RMSE: 0.31053444743156433
MAE: 0.2714950442314148
R^2: -0.01275799097278929

For Neu:
MSE: 0.08044843375682831
RMSE: 0.28363433480262756
MAE: 0.23654378950595856
R^2: -0.026453081492521013

For Agre:
MSE: 0.09927501529455185
RMSE: 0.31507939100265503
MAE: 0.27613645792007446
R^2: -0.008160997230668654

For Con:
MSE: 0.09539368748664856
RMSE: 0.3088586926460266
MAE: 0.2674645781517029
R^2: -0.04105183020338954

For Ope:
MSE: 0.10274042934179306
RMSE: 0.3205314874649048
MAE: 0.2803768813610077
R^2: -0.02278668960070407



In [ ]:

df_predictions = pd.DataFrame(np.transpose(all_predictions), columns=trait_names)


correlation_matrix_predictions = df_predictions.corr()
print("\nCorrelation matrix for predicted values:")
print(correlation_matrix_predictions)



Correlation matrix for predicted values:
           Ext       Neu      Agre       Con       Ope
Ext   1.000000  0.028796  0.015951 -0.236116  0.067094
Neu   0.028796  1.000000  0.081678 -0.081718  0.046438
Agre  0.015951  0.081678  1.000000 -0.194942  0.101301
Con  -0.236116 -0.081718 -0.194942  1.000000 -0.050647
Ope   0.067094  0.046438  0.101301 -0.050647  1.000000


In [ ]:
def color_cells(val):
    if val < 0:
        return 'background-color: red'
    elif val == 1:
        return 'background-color: yellow'
    else:
        return 'background-color: green'

styled_df = correlation_matrix_predictions.style.applymap(color_cells)

styled_df